# Generation of fixation file (.CSV)

In [1]:
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame as DF
import seaborn as sns
sns.set(color_codes=True)
#np.random.seed(sum(map(ord, "distributions")))
from sklearn import linear_model  # packages for the logistic regression function to plot the logistic regression 
from sklearn.linear_model import LogisticRegression # packages for the logistic regression function to plot the logistic regression 
import scipy
from scipy import stats, integrate
from scipy.stats import mode
from scipy.stats.stats import pearsonr # Pearson's correlation
from copy import copy as copy
import operator as operator
import pylab

# Import Fixation reports

In [2]:
path_fixat ="/Users/pradyumna/OneDrive - University College London/PhD Projects/FoodFramingValue/Reports/Fixations_FullReport_RTfix_v4.txt"
# load csv for IA and Sacc
fixat_data = [pd.read_csv(path_fixat, sep="\t")]
# Concatenate to generate frames
fixat_data = pd.concat(fixat_data)

path_mssg ="/Users/pradyumna/OneDrive - University College London/PhD Projects/FoodFramingValue/Reports/Messages_FullReport_v1.txt"
# load csv for IA and Sacc
mssg_data = [pd.read_csv(path_mssg, sep="\t")]
# Concatenate to generate frames
mssg_data = pd.concat(mssg_data)

In [3]:
mssg_data

DATA_FILE  trial      CURRENT_MSG_TEXT CURRENT_MSG_FIX_DURATION  \
0         p1.edf    145  DISPLAY_IMAGES_ONSET                      311   
1         p1.edf    145               NONE_FT                      311   
2         p1.edf    145          ALL_INVISBLE                      311   
3         p1.edf    145          LOWERLEFT_FT                      126   
4         p1.edf    145    LOWER_LEFT_VISIBLE                      126   
5         p1.edf    145  DISPLAY_IMAGES_ONSET                      126   
6         p1.edf    145         LOWERRIGHT_FT                      386   
7         p1.edf    145   LOWER_RIGHT_VISIBLE                      386   
8         p1.edf    145  DISPLAY_IMAGES_ONSET                      386   
9         p1.edf    145          LOWERLEFT_FT                      318   
10        p1.edf    145    LOWER_LEFT_VISIBLE                      318   
11        p1.edf    145  DISPLAY_IMAGES_ONSET                      318   
12        p1.edf    145               NONE_FT                      664   
13        p1.edf    145          ALL_INVISBLE                      664   
14        p1.edf    145  DISPLAY_IMAGES_ONSET                      664   
15        p1.edf    145          LOWERLEFT_FT                      176   
16        p1.edf    145    LOWER_LEFT_VISIBLE                      176   
17        p1.edf    145  DISPLAY_IMAGES_ONSET                      176   
18        p1.edf    145               NONE_FT                      122   
19        p1.edf    145          ALL_INVISBLE                      122   
20        p1.edf    145  DISPLAY_IMAGES_ONSET                      122   
21        p1.edf    145          LOWERLEFT_FT                      384   
22        p1.edf    145    LOWER_LEFT_VISIBLE                      384   
23        p1.edf    145  DISPLAY_IMAGES_ONSET                      384   
24        p1.edf    145         LOWERRIGHT_FT                      240   
25        p1.edf    145   LOWER_RIGHT_VISIBLE                      240   
26        p1.edf    145  DISPLAY_IMAGES_ONSET                      240   
27        p1.edf    145          LOWERLEFT_FT                      366   
28        p1.edf    145    LOWER_LEFT_VISIBLE                      366   
29        p1.edf    145  DISPLAY_IMAGES_ONSET                      366   
...          ...    ...                   ...                      ...   
132867   p43.edf    238  DISPLAY_IMAGES_ONSET                      280   
132868   p43.edf    238          LOWERLEFT_FT                      314   
132869   p43.edf    238    LOWER_LEFT_VISIBLE                      314   
132870   p43.edf    238  DISPLAY_IMAGES_ONSET                      314   
132871   p43.edf    238       CHOICE_RESPONSE                       76   
132872   p43.edf    238       STORE_CHOICE_LL                       76   
132873   p43.edf    239  DISPLAY_IMAGES_ONSET                      196   
132874   p43.edf    239               NONE_FT                      196   
132875   p43.edf    239          ALL_INVISBLE                      196   
132876   p43.edf    239          LOWERLEFT_FT                       76   
132877   p43.edf    239    LOWER_LEFT_VISIBLE                       76   
132878   p43.edf    239  DISPLAY_IMAGES_ONSET                       76   
132879   p43.edf    239         LOWERRIGHT_FT                      312   
132880   p43.edf    239   LOWER_RIGHT_VISIBLE                      312   
132881   p43.edf    239  DISPLAY_IMAGES_ONSET                      312   
132882   p43.edf    239          LOWERLEFT_FT                      290   
132883   p43.edf    239    LOWER_LEFT_VISIBLE                      290   
132884   p43.edf    239  DISPLAY_IMAGES_ONSET                      290   
132885   p43.edf    239         LOWERRIGHT_FT                      274   
132886   p43.edf    239   LOWER_RIGHT_VISIBLE                      274   
132887   p43.edf    239  DISPLAY_IMAGES_ONSET                      274   
132888   p43.edf    239       CHOICE_RESPONSE                      274   
132889   p4

## Eliminate excluded participants
### remember due to blinking p40.edf was eliminated at the beginning!

In [4]:
Part2Delete = ['p40.edf']
for i in Part2Delete:
    print(i)
    fixat_data = fixat_data[(fixat_data.DATA_FILE != i)]
    mssg_data = mssg_data[(mssg_data.DATA_FILE != i)]

p40.edf


In [5]:
participant = fixat_data['DATA_FILE'].unique()
participant_names = fixat_data['DATA_FILE'].unique()

#Extract just names of participant (for folder name in future steps)
yy=0
for xx in participant_names:
    participant_names[yy]=xx.rstrip(".edf")
    yy+=1
print (participant_names)

trial_n = fixat_data['trial'].unique()

print("Number of participants:", len(participant))
print ("Number of trials:", len(trial_n))

['p1' 'p5' 'p6' 'p7' 'p8' 'p9' 'p10' 'p11' 'p12' 'p13' 'p14' 'p15' 'p16'
 'p17' 'p18' 'p19' 'p20' 'p21' 'p22' 'p23' 'p24' 'p25' 'p26' 'p27' 'p28'
 'p29' 'p30' 'p31' 'p32' 'p33' 'p34' 'p35' 'p36' 'p37' 'p38' 'p39' 'p41'
 'p42' 'p43']
Number of participants: 39
Number of trials: 240


In [6]:
names_frame = pd.DataFrame({'OGName':participant,'Numeration':range(1,40)})
names_frame

OGName  Numeration
0    p1.edf           1
1    p5.edf           2
2    p6.edf           3
3    p7.edf           4
4    p8.edf           5
5    p9.edf           6
6   p10.edf           7
7   p11.edf           8
8   p12.edf           9
9   p13.edf          10
10  p14.edf          11
11  p15.edf          12
12  p16.edf          13
13  p17.edf          14
14  p18.edf          15
15  p19.edf          16
16  p20.edf          17
17  p21.edf          18
18  p22.edf          19
19  p23.edf          20
20  p24.edf          21
21  p25.edf          22
22  p26.edf          23
23  p27.edf          24
24  p28.edf          25
25  p29.edf          26
26  p30.edf          27
27  p31.edf          28
28  p32.edf          29
29  p33.edf          30
30  p34.edf          31
31  p35.edf          32
32  p36.edf          33
33  p37.edf          34
34  p38.edf          35
35  p39.edf          36
36  p41.edf          37
37  p42.edf          38
38  p43.edf          39

In [7]:
# From BlinkMatrix (from FoodFrameAnalysis.ipnyb) we find that we have to take out participants

# In terms of particpant numeration the participants to be elimnated are 
# 6, 7, 17, 25, 30, 33,37,38

Part2Delete = ['p9.edf' , 'p10.edf' , 'p20.edf' , 'p28.edf', 'p33.edf' , 'p36.edf','p41.edf','p42.edf']
for i in Part2Delete:
    print(i)
    fixat_data = fixat_data[(fixat_data.DATA_FILE != i)]
    mssg_data = mssg_data[(mssg_data.DATA_FILE != i)]
    names_frame = names_frame[names_frame.OGName != i]

participant =  fixat_data["DATA_FILE"].unique()

print(fixat_data["DATA_FILE"].unique())
len(fixat_data["DATA_FILE"].unique())


p9.edf
p10.edf
p20.edf
p28.edf
p33.edf
p36.edf
p41.edf
p42.edf
['p1.edf' 'p5.edf' 'p6.edf' 'p7.edf' 'p8.edf' 'p11.edf' 'p12.edf'
 'p13.edf' 'p14.edf' 'p15.edf' 'p16.edf' 'p17.edf' 'p18.edf' 'p19.edf'
 'p21.edf' 'p22.edf' 'p23.edf' 'p24.edf' 'p25.edf' 'p26.edf' 'p27.edf'
 'p29.edf' 'p30.edf' 'p31.edf' 'p32.edf' 'p34.edf' 'p35.edf' 'p37.edf'
 'p38.edf' 'p39.edf' 'p43.edf']


31

In [8]:
# I used 0 for blank fixations, 1 for left, 2 for right, 3 for center.
names_frame

OGName  Numeration
0    p1.edf           1
1    p5.edf           2
2    p6.edf           3
3    p7.edf           4
4    p8.edf           5
7   p11.edf           8
8   p12.edf           9
9   p13.edf          10
10  p14.edf          11
11  p15.edf          12
12  p16.edf          13
13  p17.edf          14
14  p18.edf          15
15  p19.edf          16
17  p21.edf          18
18  p22.edf          19
19  p23.edf          20
20  p24.edf          21
21  p25.edf          22
22  p26.edf          23
23  p27.edf          24
25  p29.edf          26
26  p30.edf          27
27  p31.edf          28
28  p32.edf          29
30  p34.edf          31
31  p35.edf          32
33  p37.edf          34
34  p38.edf          35
35  p39.edf          36
38  p43.edf          39

In [9]:
i=0
fixations = []
#Run for each individual participant
for i in range(len(participant)):
    trial_n = 240
    for  j in range(trial_n):
        fixat_data_trial=fixat_data.loc[(fixat_data['DATA_FILE']==participant[i]) & (fixat_data['trial']== j+1) & (fixat_data['trial_type']== 'Experiment')]
        mssg_data_trial=mssg_data.loc[(mssg_data['DATA_FILE']==participant[i]) & (mssg_data['trial']== j+1) & (mssg_data['trial_type']== 'Experiment')]

        fixat_n = len(fixat_data_trial)
                
        current_trial_time = 0

        trial_start_time = mssg_data_trial.iloc[0].CURRENT_MSG_TIME
        
        #if first fixation is at the center or outside the area of the items we consider the trial start with the blank fixation
        if fixat_data_trial.iloc[0].CURRENT_FIX_INTEREST_AREA_ID == '.' or fixat_data_trial.iloc[0].CURRENT_FIX_INTEREST_AREA_ID == '4':
            trial_fix_init = mssg_data_trial.iloc[0].CURRENT_MSG_TIME
        #if first fixation is over one of the items we consider the trial start with that item
        if fixat_data_trial.iloc[0].CURRENT_FIX_INTEREST_AREA_ID == '1' or fixat_data_trial.iloc[0].CURRENT_FIX_INTEREST_AREA_ID == '2':
            #we assume center fixation since the beginning of the trial
            parcode_fix = names_frame.Numeration.values[i]
            trial_fix = fixat_data_trial.iloc[0].trial - 1
            fix_time_fix = fixat_data_trial.iloc[0].CURRENT_FIX_START - mssg_data_trial.iloc[0].CURRENT_MSG_TIME
            block_fix =fixat_data_trial.iloc[0].Block_Cond
            fix_item_fix =  3
            
            #Extract reaction time for that trial (only for test)
            RT_fix_test = fixat_instance_data.CHOICE_RT 
            part_name_fix_test = fixat_instance_data.DATA_FILE 
            
            
            fixations.append([part_name_fix_test,parcode_fix, trial_fix, fix_item_fix, fix_time_fix,RT_fix_test,block_fix]) 
            #and we reset the fixation for the trial
            trial_fix_init = fixat_data_trial.iloc[0].CURRENT_FIX_START
        
        
        for  k in range(fixat_n):

            # extract saccade data for this participant
            fixat_instance_data = fixat_data_trial.iloc[k]
            parcode_fix = names_frame.Numeration.values[i]
            trial_fix = fixat_instance_data.trial - 1 # remember in the aDDM_toolbox format it starts from zero. 
            block_fix =fixat_instance_data.Block_Cond


            #Considered the fixation duration as the end of the previous fixations and 
            #the end of current fixation. In case of the first fixation, we consider,
            # the beginning of the trial as the start, in case we start in blank
            
                        
            #Include condition to define fix_time_fix considering that we have the last fixation
            if k == fixat_n-1:  # if this is the last element
                #calculate the expected time of trial ending considering RT as the real length 
                RT_final_time = fixat_instance_data.CHOICE_RT + trial_start_time 
                fix_time_fix = RT_final_time - trial_fix_init

            # if not, calculate time for fixation as the time between the end of  previous fixation
            # and the end of current fixatoon
            
            
            
            ## Areas defined in DataViewer Reports is 
            ## 1 : Left
            ## 2 : Right
            ## 3 : Symbol for block
            ## 4 : Center space
            ## Any other area will be indicated as empty
            
            
            else:
                fix_time_fix = fixat_instance_data.CURRENT_FIX_END - trial_fix_init
            
            if fixat_instance_data.CURRENT_FIX_INTEREST_AREA_ID == '1': 
                fix_item_fix =  1
            elif fixat_instance_data.CURRENT_FIX_INTEREST_AREA_ID == '2': 
                fix_item_fix =  2
            elif fixat_instance_data.CURRENT_FIX_INTEREST_AREA_ID == '.': 
                fix_item_fix =  0
            elif fixat_instance_data.CURRENT_FIX_INTEREST_AREA_ID == '4' and k==0: 
            # we assign center fixation only if it's the first one 
                fix_item_fix =  3    
            else  :
                fix_item_fix =  0   
                
            #Extract reaction time for that trial (only for test)
            RT_fix_test = fixat_instance_data.CHOICE_RT 
            part_name_fix_test = fixat_instance_data.DATA_FILE 
            
            fixations.append([part_name_fix_test,parcode_fix, trial_fix, fix_item_fix, fix_time_fix,RT_fix_test,block_fix]) 
            
            #update the init to the end of the current fixation
            trial_fix_init = fixat_instance_data.CURRENT_FIX_END


In [10]:
fixation_file = pd.DataFrame(fixations,columns=['name','parcode', 'trial', 'fix_item', 'fix_time','rt','block'])
dataframe_long = len(fixation_file)

In [11]:
fixation_file

name  parcode  trial  fix_item  fix_time    rt  block
0       p1.edf        1      0         0       437  3327    2.0
1       p1.edf        1      0         0       164  3327    2.0
2       p1.edf        1      0         1       302  3327    2.0
3       p1.edf        1      0         1       198  3327    2.0
4       p1.edf        1      0         1       378  3327    2.0
5       p1.edf        1      0         2       432  3327    2.0
6       p1.edf        1      0         2       262  3327    2.0
7       p1.edf        1      0         1       358  3327    2.0
8       p1.edf        1      0         1       358  3327    2.0
9       p1.edf        1      0         1       438  3327    2.0
10      p1.edf        1      1         3       215  3424    2.0
11      p1.edf        1      1         1       338  3424    2.0
12      p1.edf        1      1         1       348  3424    2.0
13      p1.edf        1      1         2       740  3424    2.0
14      p1.edf        1      1         2       748  3424    2.0
15      p1.edf        1      1         1       352  3424    2.0
16      p1.edf        1      1         1       280  3424    2.0
17      p1.edf        1      1         2       292  3424    2.0
18      p1.edf        1      1         2       111  3424    2.0
19      p1.edf        1      2         3       339  3691    2.0
20      p1.edf        1      2         1       116  3691    2.0
21      p1.edf        1      2         1       240  3691    2.0
22      p1.edf        1      2         1       198  3691    2.0
23      p1.edf        1      2         1       328  3691    2.0
24      p1.edf        1      2         1       226  3691    2.0
25      p1.edf        1      2         2       424  3691    2.0
26      p1.edf        1      2         2       182  3691    2.0
27      p1.edf        1      2         2       284  3691    2.0
28      p1.edf        1      2         2       168  3691    2.0
29      p1.edf        1      2         2       610  3691    2.0
...        ...      ...    ...       ...       ...   ...    ...
76274  p43.edf       39    237         2       262  3394    1.0
76275  p43.edf       39    237         2       302  3394    1.0
76276  p43.edf       39    237         2       214  3394    1.0
76277  p43.edf       39    237         0       112  3394    1.0
76278  p43.edf       39    237         1       356  3394    1.0
76279  p43.edf       39    237         1       125  3394    1.0
76280  p43.edf       39    238         3       194  5280    1.0
76281  p43.edf       39    238         1       106  5280    1.0
76282  p43.edf       39    238         1       254  5280    1.0
76283  p43.edf       39    238         1       294  5280    1.0
76284  p43.edf       39    238         1       424  5280    1.0
76285  p43.edf       39    238         2       356  5280    1.0
76286  p43.edf       39    238         2       666  5280    1.0
76287  p43.edf       39    238         2       638  5280    1.0
76288  p43.edf       39    238         2       544  5280    1.0
76289  p43.edf       39    238         2       174  5280    1.0
76290  p43.edf       39    238         2       422  5280    1.0
76291  p43.edf       39    238         1       360  5280    1.0
76292  p43.edf       39    238         1       192  5280    1.0
76293  p43.edf       39    238         1       316  5280    1.0
76294  p43.edf       39    238         2       340  5280    1.0
76295  p43.edf       39    239         3         0  5280    1.0
76296  p43.edf       39    239         2        26  1905    1.0
76297  p43.edf       39    239         2       166  1905    1.0
76298  p43.edf       39    239         2       258  1905    1.0
76299  p43.edf       39    239         2       400  1905    1.0
76300  p43.edf       39    239         2       136  1905    1.0
76301  p43.edf       39    239         1       284  1905    1.0
76302  p43.edf       39    239         1       178  1905    1.0
76303  p43.edf       39    239         1       457  1905    1.0

[76304 rows x 7 columns]

## Count microsaccades : fixations within the area

In [12]:
fixation_file.parcode.unique()

array([ 1,  2,  3,  4,  5,  8,  9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20,
       21, 22, 23, 24, 26, 27, 28, 29, 31, 32, 34, 35, 36, 39])

In [13]:
microfixations_dat = []
for i in fixation_file.parcode.unique():
    for j in fixation_file.trial.unique():
        fixations_part_trial = fixation_file.loc[(fixation_file['parcode'] == i) & (fixation_file['trial'] == j)] 
        
        part = fixations_part_trial.parcode.unique()[0]
        trial_n = fixations_part_trial.trial.unique()[0]
        
        microfix_1 =  fixations_part_trial.loc[(fixations_part_trial['fix_item'] == 1)] 
        microfix_2 =  fixations_part_trial.loc[(fixations_part_trial['fix_item'] == 2)] 
        
        microfix_1_num = microfix_1.shape[0]
        microfix_2_num = microfix_2.shape[0]
        
        
        # Eliminate first fixations 
        previous_item = -1
        to_eliminate = []
        for  a  in  range(len(fixations_part_trial)):
                if fixations_part_trial.iloc[a].fix_item != previous_item:
                    to_eliminate.append(fixations_part_trial.iloc[a].name)
                previous_item = fixations_part_trial.iloc[a].fix_item
                    
        fixations_part_trial_dropped = fixations_part_trial.drop(to_eliminate)
        
        microfix_1_drop =  fixations_part_trial_dropped.loc[(fixations_part_trial_dropped['fix_item'] == 1)] 
        microfix_2_drop =  fixations_part_trial_dropped.loc[(fixations_part_trial_dropped['fix_item'] == 2)] 
        
        microfix_1_drop_num = microfix_1_drop.shape[0]
        microfix_2_drop_num = microfix_2_drop.shape[0]
        microfix_1_drop_timeavg = microfix_1_drop.fix_time.mean()
        microfix_2_drop_timeavg = microfix_2_drop.fix_time.mean()
        
        
        microfixations_dat.append([part, trial_n, microfix_1_num, microfix_2_num ,microfix_1_drop_num ,microfix_2_drop_num,microfix_1_drop_timeavg,microfix_2_drop_timeavg]) 

microfixations_file = pd.DataFrame(microfixations_dat,columns=['parcode', 'trial', 'microfix1', 'microfix2', 'microfix1_drop', 'microfix2_drop','microfix_1_drop_timeavg','microfix_2_drop_timeavg'])

In [14]:
microfixations_file

parcode  trial  microfix1  microfix2  microfix1_drop  microfix2_drop  \
0           1      0          6          2               4               1   
1           1      1          4          4               2               2   
2           1      2          5          7               4               6   
3           1      3         12          9               8               7   
4           1      4         14          6              10               4   
5           1      5         10          2               7               0   
6           1      6          3          4               2               3   
7           1      7          8         10               6               8   
8           1      8          6          5               4               3   
9           1      9          2          5               1               4   
10          1     10          9          4               7               3   
11          1     11          5          3               3               2   
12          1     12          5          5               3               3   
13          1     13         15         15              11              10   
14          1     14          6          3               4               2   
15          1     15         11         10               8               7   
16          1     16          4          8               2               6   
17          1     17          4          6               3               5   
18          1     18          8          9               5               7   
19          1     19          2          6               1               5   
20          1     20          5          2               3               1   
21          1     21          7          5               5               4   
22          1     22          6          6               4               4   
23          1     23          3          7               2               6   
24          1     24          5          4               3               3   
25          1     25          8          4               5               2   
26          1     26          3          3               2               2   
27          1     27          4          6               2               4   
28          1     28         11          8               8               6   
29          1     29          4          4               2               3   
...       ...    ...        ...        ...             ...             ...   
7410       39    210          5          7               3               5   
7411       39    211          4          4               3               2   
7412       39    212          5          5               3               3   
7413       39    213          7          4               5               2   
7414       39    214          4          5               2               3   
7415       39    215          2          4               1               3   
7416       39    216          4          6               3               4   
7417       39    217          5          4               4               3   
7418       39    218          3          2               2               1   
7419       39    219          3          3               2               1   
7420       39    220          5          2               4               1   
7421       39    221          2          2               1               1   
7422       39    222          6          6               4               4   
7423       39    223         16         11              12               9   
7424       39    224          9          8               7               6   
7425       39    225          3          2               2               1   
7426       39    226          5          2               3               1   
7427       39    227          8          5               6               2   
7428       39    228          2          6               1               4   
7429       39    229        

In [15]:
microfixations_file.to_csv("Output/Microfixations_AllConditions_31.csv",index = False)

## ####################   END  ################ #